# Drive mount

In [12]:
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install datasets
!pip install transformers[torch] -U

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple/

[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
zsh:1: no matches found: transformers[torch]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
import os
import json
import numpy as np
import pandas as pd
import re
import string
from collections import Counter
from tqdm import tqdm

import torch
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForCausalLM,
)
from datasets import load_dataset, Dataset, DatasetDict
from accelerate import Accelerator


# 데이터를 Colab 서버 내에 세팅

In [14]:
# open.zip 파일을 업로드한 경로를 확인하여 적용
!unzip  "/content/drive/MyDrive/2024인하대/open.zip"

unzip:  cannot find or open /content/drive/MyDrive/2024인하대/open.zip, /content/drive/MyDrive/2024인하대/open.zip.zip or /content/drive/MyDrive/2024인하대/open.zip.ZIP.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# Load Validation / Test dataset

In [15]:
file_path = './data/train.csv'
train_data = pd.read_csv(file_path)

train_data = train_data.sample(frac=1).reset_index(drop=True)

val_data=train_data[:10]
train_data = train_data[10:]

val_label_df = val_data[['question', 'answer']]

train_data.head(2)

,id,context,question,answer
10,TRAIN_15560,소속 직원들의 미공개 정보를 이용한 신도시 부동산 투기로 논란에 직면했던 한국토지주...,LH는 무슨 의심을 투기 물의 이후에 받고 있어,용역 입찰담합
11,TRAIN_15936,"수천만원까지 법원 공탁금을 보유하고 있으면서도 과태료, 과징금, 부담금, 이행강제금...",수원시 소재 A법인이 납부하지 않은 과태료 액수는,1억8천만원


# Model Load

In [16]:
# llama-2-ko-7b 모델 로드
model_id = "beomi/Llama-3-Open-Ko-8B"
model = AutoModelForCausalLM.from_pretrained(model_id,
                                            torch_dtype="auto") # 한정된 메모리에서 사용하기 위해, 4비트 정밀도로 로드

tokenizer = AutoTokenizer.from_pretrained(model_id) # 본 모델의 토크나이저 로드
tokenizer.use_default_system_prompt = False  # 기본 시스템 프롬프트를 사용하지 않도록 설정

accelerator = Accelerator()

model, tokenizer = accelerator.prepare(model, tokenizer)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


RuntimeError: MPS backend out of memory (MPS allocated: 18.05 GB, other allocations: 384.00 KB, max allowed: 18.13 GB). Tried to allocate 112.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

# Define F1 score

In [ ]:
def normalize_answer(s):
    def remove_(text):
        ''' 불필요한 기호 제거 '''
        text = re.sub("'", " ", text)
        text = re.sub('"', " ", text)
        text = re.sub('《', " ", text)
        text = re.sub('》', " ", text)
        text = re.sub('<', " ", text)
        text = re.sub('>', " ", text)
        text = re.sub('〈', " ", text)
        text = re.sub('〉', " ", text)
        text = re.sub("\(", " ", text)
        text = re.sub("\)", " ", text)
        text = re.sub("‘", " ", text)
        text = re.sub("’", " ", text)
        return text

    def white_space_fix(text):
        '''연속된 공백일 경우 하나의 공백으로 대체'''
        return ' '.join(text.split())

    def remove_punc(text):
        '''구두점 제거'''
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        '''소문자 전환'''
        return text.lower()

    return white_space_fix(remove_punc(lower(remove_(s))))

def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()

    # 문자 단위로 f1-score를 계산 합니다.
    prediction_Char = []
    for tok in prediction_tokens:
        now = [a for a in tok]
        prediction_Char.extend(now)

    ground_truth_Char = []
    for tok in ground_truth_tokens:
        now = [a for a in tok]
        ground_truth_Char.extend(now)

    common = Counter(prediction_Char) & Counter(ground_truth_Char)
    num_same = sum(common.values())
    if num_same == 0:
        return 0

    precision = 1.0 * num_same / len(prediction_Char)
    recall = 1.0 * num_same / len(ground_truth_Char)
    f1 = (2 * precision * recall) / (precision + recall)

    return f1

def evaluate(ground_truth_df, predictions_df):
    predictions = dict(zip(predictions_df['question'], predictions_df['answer']))
    f1 = exact_match = total = 0

    for index, row in ground_truth_df.iterrows():
        question_text = row['question']
        ground_truths = row['answer']
        total += 1
        if question_text not in predictions:
            continue
        prediction = predictions[question_text]
        f1 = f1 + f1_score(prediction, ground_truths)

    f1 = 100.0 * f1 / total
    return {'f1': f1}


# Validation
- 로드한 모델에 대해 간단한 프롬프트를 입력하여 성능 검증

In [ ]:
qa_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
# 모델로 추론 후, 전처리를 수행한 뒤, 완성된 정답으로 반환합니다.
def generate_response(question_prompt):
    # 생성할 최대 토큰 수와, 답변 생성 수, 패딩 토큰의 idx를 지정하여 모델 파이프 라인을 설정하고, 답변을 생성합니다.
    response = qa_pipeline(question_prompt, max_new_tokens=50, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)[0]['generated_text']
    if "Answer:" in response:
            # Answer: 이후에 생성된 토큰 들만을 답변으로 사용합니다.
            response = response.split("Answer:", 1)[1][:20]

            # 토큰 반복 생성 및 노이즈 토큰 관련 처리
            if "Que" in response:
                response = response.split("Que", 1)[0]
            if "⊙" in response:
                response = response.split("⊙", 1)[0]
            if "Con" in response:
                response = response.split("Con", 1)[0]
    return response

In [ ]:
#Google Colab T4 GPU 기준 10개 샘플 답안 생성에 1분 소요
predict_dict = {}
count = 0

for index, row in val_data.iterrows():
    try:
        context = row['context']
        question = row['question']


        if context is not None and question is not None:
            question_prompt = f"너는 주어진 Context를 토대로 Question에 답하는 챗봇이야. \
                                Question에 대한 답변만 가급적 한 단어로 최대한 간결하게 답변하도록 해. \
                                Context: {context} Question: {question}\n Answer:"

            answer = generate_response(question_prompt)
            predict_dict[question] = answer
        else:
            predict_dict[question] = 'Invalid question or context'

        print("Answer for question:", question, ":", predict_dict[question])
        count += 1
        print("Processed count:", count)
    except Exception as e:
        print(f"Error processing question {e}")



In [ ]:
val_inf_df = pd.DataFrame(list(predict_dict.items()), columns=['question', 'answer'])
val_inf_df.head()

In [ ]:
# f1-score 를 출력합니다.
results = evaluate( val_inf_df, val_label_df)
print(results)

# Inference

In [ ]:
file_path = '/content/test.csv'
test_data = pd.read_csv(file_path)

In [ ]:
# 모델 추론
submission_dict = {}

for index, row in test_data.iterrows():
    try:
        context = row['context']
        question = row['question']
        id = row['id']

        if context is not None and question is not None:
            question_prompt = f"너는 주어진 Context를 토대로 Question에 답하는 챗봇이야. \
                                Question에 대한 답변만 가급적 한 단어로 최대한 간결하게 답변하도록 해. \
                                Context: {context} Question: {question}\n Answer:"

            answer = generate_response(question_prompt)
            submission_dict[id] = answer
        else:
            submission_dict[id] = 'Invalid question or context'

    except Exception as e:
        print(f"Error processing question {e}")

# Submission

In [ ]:
# 제출
df = pd.DataFrame(list(submission_dict.items()), columns=['id', 'answer'])
df.to_csv( '/content/submission.csv', index=False)